<h1 align=center><font size = 5>Incorporating Neighbourhood Geospatial Coordinates into Toronto Dataframe.</font></h1>

# Introduction

This lab builds on the previous lab by adding latitude and longitude of each neighbourhood to the initially dataframe. 
#### This incorporation starts in the <a href="#Geospatial-Coordinates-Data-Incorporation">11th code cell</a>.

Importing required libraries...

In [1]:
from bs4 import BeautifulSoup #to aid web scraping
import requests #
import pandas as pd

Download website's html doc for scraping

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

data = requests.get(url).text

Create Soup object and extract table data

In [3]:
soup = BeautifulSoup(data, 'html5lib')
table = soup.table
tableData = table.find_all('td')

Below, I made a list (of strings in each table data) for each table data and then made a list of these lists. 

In [4]:
#make rows of desired dataframe into a list and append all into a  general list
data = []
for ind in range(len(tableData)):
    rows = []
    for string in tableData[ind].stripped_strings:
        rows.append(repr(string))
    data.append(rows)
data[0:5]

[["'M1A'", "'Not assigned'"],
 ["'M2A'", "'Not assigned'"],
 ["'M3A'", "'North York'", "'('", "'Parkwoods'", "')'"],
 ["'M4A'", "'North York'", "'('", "'Victoria Village'", "')'"],
 ["'M5A'",
  "'Downtown Toronto'",
  "'('",
  "'Regent Park'",
  "'/'",
  "'Harbourfront'",
  "')'"]]

I noticed that lists, with borough == "Not assigned", were of length 2. So I retained all lists with length > 2.

In [5]:
#retain list of all data without a borough = "Not assigned"
dataFiltered = []
for ind in range(len(data)):
    if len(data[ind]) > 2:
        dataFiltered.append(data[ind])

dataFiltered[0:5]

[["'M3A'", "'North York'", "'('", "'Parkwoods'", "')'"],
 ["'M4A'", "'North York'", "'('", "'Victoria Village'", "')'"],
 ["'M5A'",
  "'Downtown Toronto'",
  "'('",
  "'Regent Park'",
  "'/'",
  "'Harbourfront'",
  "')'"],
 ["'M6A'",
  "'North York'",
  "'('",
  "'Lawrence Manor'",
  "'/'",
  "'Lawrence Heights'",
  "')'"],
 ["'M7A'", '"Queen\'s Park"', "'(Ontario Provincial Government)'"]]

Below, I firstly removed the redundant quotes from each element.

Next, assuming all 3rd elements to the last element in each respective list was a Neighbourhood data, I concatenated all 3rd elements to the last elements in each list.

In [6]:
#delete apostrophe
for ind in range(len(dataFiltered)):
    for ind2 in range(len(dataFiltered[ind])):
        dataFiltered[ind][ind2] = dataFiltered[ind][ind2].replace("'","")

#make each list a length of 3 by concatenating all elements except the first two  
for ind in range (len(dataFiltered)):
    dataFiltered[ind] = dataFiltered[ind][0:2] + [''.join(dataFiltered[ind][2:(len(dataFiltered[ind]))])]

dataFiltered[0:5]

[['M3A', 'North York', '(Parkwoods)'],
 ['M4A', 'North York', '(Victoria Village)'],
 ['M5A', 'Downtown Toronto', '(Regent Park/Harbourfront)'],
 ['M6A', 'North York', '(Lawrence Manor/Lawrence Heights)'],
 ['M7A', '"Queens Park"', '(Ontario Provincial Government)']]

Converting to dataframe...

In [7]:
neighFrame = pd.DataFrame(dataFiltered)
neighFrame.columns = ["Postal Code", "Borough", "Neighbourhood"]
neighFrame

,Postal Code,Borough,Neighbourhood
0,M3A,North York,(Parkwoods)
1,M4A,North York,(Victoria Village)
2,M5A,Downtown Toronto,(Regent Park/Harbourfront)
3,M6A,North York,(Lawrence Manor/Lawrence Heights)
4,M7A,"""Queens Park""",(Ontario Provincial Government)
...,...,...,...
98,M8X,Etobicoke,(The Kingsway/ Montgomery Road /Old MillNorth)
99,M4Y,Downtown Toronto,(Church and Wellesley)
100,M7Y,East Toronto,Business reply mailProcessing Centre969 Easter...
101,M8Y,Etobicoke,"(Old Mill""South / Kings Mill Park /""Sunnylea/H..."


Cleaning Data...

In [8]:
#delete all double quotes
for column in neighFrame.columns:
    neighFrame[column] = neighFrame[column].str.replace('"','')

#delete all brackets in "Neighbourhood" column data starting with "("
colStartingWthBrack = neighFrame[neighFrame["Neighbourhood"].str.startswith("(")]
#remove ")"
neighFrame.loc[colStartingWthBrack.index,"Neighbourhood"] = \
neighFrame.loc[colStartingWthBrack.index,"Neighbourhood"].str.replace(')','')
#remove "("
neighFrame.loc[colStartingWthBrack.index,"Neighbourhood"] = \
neighFrame.loc[colStartingWthBrack.index,"Neighbourhood"].str.replace('(','')

#delete "Business reply mail" from Neighbourhood column row 100
neighFrame.loc[100, "Neighbourhood"] = \
neighFrame.loc[100, "Neighbourhood"][(len("Business reply mail")):]

#change all "/" to ", "
neighFrame["Neighbourhood"] = neighFrame["Neighbourhood"].str.replace('/',', ')

#change all " ," to ","
neighFrame["Neighbourhood"] = neighFrame["Neighbourhood"].str.replace(' ,',',')

#change all "  " to " "
neighFrame["Neighbourhood"] = neighFrame["Neighbourhood"].str.replace('  ',' ')

In [9]:
neighFrame

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queens Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old MillNorth"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Processing Centre969 Eastern(Enclave of M4L)
101,M8Y,Etobicoke,"Old MillSouth, Kings Mill Park, Sunnylea, Humb..."


In [10]:
neighFrame.shape

(103, 3)

# Geospatial Coordinates Data Incorporation

In [11]:
path = r"C:\Users\ADESOYE\OneDrive\Coursera\Course 10 Capstone Project\Geospatial_Coordinates.csv"

In [12]:
#read file into pandas dataframe
geoData = pd.read_csv(path)
geoData

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
#Sorting both Geo data and neigbouhood Dataframe by Postal Code...
geoData.sort_values(by=['Postal Code'], inplace=True)
neighFrame.sort_values(by=['Postal Code'], inplace=True )
neighFrame.reset_index(drop = True, inplace= True)

In [14]:
#Defining latitude and longitude variables 
latitude, longitude= geoData["Latitude"],geoData["Longitude"]

#Inserting them in the main data frame...
neighFrame["Latitude"], neighFrame["Longitude"] = latitude, longitude

In [15]:
neighFrame

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
